1. [Notebook goal](#notebookgoal)
2. [PC What ?](#pca)
3. [Let's code](#letscode)
4. [PCA with sklearn](#sklearn)
5. [Bonus](#bonus)

# 1. Notebook goal <a name="notebookgoal"></a>
In this notebook we'll focus on implementing PCA  with little help possible from sklearn; You'll see how easy it is; And don't worry we'll compare the results ;)

# 2. PC What ? <a name="pca"></a>
Principal component analysis (PCA) allows us to summarize and to visualize the information in a data set containing individuals/observations described by multiple inter-correlated quantitative variables. Each variable could be considered as a different dimension. If you have more than 3 variables in your data sets, it could be very difficult to visualize a multi-dimensional hyperspace.

Principal component analysis is used to extract the important information from a multivariate data table and to express this information as a set of few new variables called principal components. These new variables correspond to a linear combination of the originals. The number of principal components is less than or equal to the number of original variables.

The information in a given data set corresponds to the total variation it contains. The goal of PCA is to identify directions (or principal components) along which the variation in the data is maximal.

In other words, PCA reduces the dimensionality of a multivariate data to two or three principal components, that can be visualized graphically, with minimal loss of information [[sthda](http://www.sthda.com/english/articles/31-principal-component-methods-in-r-practical-guide/112-pca-principal-component-analysis-essentials/)].

# 2. Let's code <a name="letscode"></a> 

These are the libraries we need, nothing special ¯\\\_(ツ)\_/¯

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv("/kaggle/input/iris-flower-dataset/IRIS.csv")

In [ ]:
df

In [ ]:
df.describe()

In [ ]:
sns.pairplot(df, hue = 'species')

In [ ]:
# We're seperating the species column
species = df["species"].tolist()
X = df.drop("species", 1)

In [ ]:
# Standardize the data
X = (X - X.mean()) / X.std(ddof=0)

**Note**: instead of standardizing the data manually, we can use sklearns StandardScaler 
> StandardScaler().fit_transform(X)

In [ ]:
# Calculating the correlation matrix of the data
X_corr = (1 / 150) * X.T.dot(X)

**Note**: instead of calculating the correlation matrix manually we can just use 
> X.corr()

In [ ]:
# Plotting the correlation matrix
plt.figure(figsize=(10,10))
sns.heatmap(X_corr, vmax=1, square=True,annot=True)
plt.title('Correlation matrix')

The next step is to extract the eign values and their respective eigen vectors.
(
It's important to have the sorted list of eigen values / eigen vectors) .

We'll try to calculate them using two methods :

In [ ]:
# method1
u,s,v = np.linalg.svd(X_corr)
eig_values, eig_vectors = s, u
eig_values, eig_vectors

In [ ]:
# method2
np.linalg.eig(X_corr)

Notice how the direction of some vectors is not the same in the two outputs. Despite that,our analysis will [still be valid](https://stats.stackexchange.com/questions/88880/does-the-sign-of-scores-or-of-loadings-in-pca-or-fa-have-a-meaning-may-i-revers)

In [ ]:
np.sum(eig_values)

Wooooooo, the sum of the eigen values is equal to number of variables. we're going in the right direction ;)

In [ ]:
# plotting the variance explained by each PC 
explained_variance=(eig_values / np.sum(eig_values))*100
plt.figure(figsize=(8,4))
plt.bar(range(4), explained_variance, alpha=0.6)
plt.ylabel('Percentage of explained variance')
plt.xlabel('Dimensions')

This bar chart shows us that the first two dimensions will be enough to represent the data.

So let's find these principal components:

In [ ]:
# calculating our new axis
pc1 = X.dot(eig_vectors[:,0])
pc2 = X.dot(eig_vectors[:,1])

In [ ]:
# plotting in 2D
def plot_scatter(pc1, pc2):
    fig, ax = plt.subplots(figsize=(15, 8))
    
    species_unique = list(set(species))
    species_colors = ["r","b","g"]
    
    for i, spec in enumerate(species):
        plt.scatter(pc1[i], pc2[i], label = spec, s = 20, c=species_colors[species_unique.index(spec)])
        ax.annotate(str(i+1), (pc1[i],pc2[i]))
    
    from collections import OrderedDict
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = OrderedDict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys(), prop={'size': 15}, loc=4)
    
    ax.set_xlabel('Principal Component 1', fontsize = 15)
    ax.set_ylabel('Principal Component 2', fontsize = 15)
    ax.axhline(y=0, color="grey", linestyle="--")
    ax.axvline(x=0, color="grey", linestyle="--")
    
    plt.grid()
    plt.axis([-4, 4, -3, 3])
    plt.show()
    
plot_scatter(pc1, pc2)

In [ ]:
def plot_correlation_circle(pc1, pc2):    
    fig, ax = plt.subplots(figsize=(16, 10))

    for i in range(X.shape[1]):
        x = np.corrcoef(pc1,X[X.columns[i]])[0,1]
        y = np.corrcoef(pc2,X[X.columns[i]])[0,1]
        ax.annotate("", xy= (x,y), xytext=(0, 0),arrowprops=dict(arrowstyle="->"))
        ax.annotate(X.columns[i], (x+0.02,y+0.02), size=12)


    ax.set_title('Correlation circle')
    ax.axhline(y=0, color="grey", linestyle="--")
    ax.axvline(x=0, color="grey", linestyle="--")

    an = np.linspace(0, 2 * np.pi, 100)
    plt.plot(np.cos(an), np.sin(an))
    plt.axis('equal')
    plt.show()
    
plot_correlation_circle(pc1,pc2)

Well it seems that:
* petal length & petal width are the main things that characterizes the data
* virginica has relatively bigger petals while setosa has the smallest
* a small group of setosa have a relatively big sepal_width

# 4. PCA with sklearn <a name="sklearn"></a>

In this section we'll use the famous sklearn library instead

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
X = df.drop("species", 1)
X = StandardScaler().fit_transform(X)
pca = PCA()
result = pca.fit_transform(X)
# Remember what we said about the sign of eigen vectors that might change ?
pc1 = - result[:,0]
pc2 = - result[:,1]
plot_scatter(pc1, pc2)

The same plot, isn't it ?

# 4. Bonus <a name="bonus"></a>

This Free DLC consists of plotting the data in 3D.

Remember that in our case plotting the data in 2D is sufficient.

In [ ]:
import plotly.express as px

In [ ]:
pc3 = result[:,2]

In [ ]:
pcs = pd.DataFrame(list(zip(pc1, pc2, pc3, species)),columns =['pc1', 'pc2', 'pc3', 'species']) 
fig = px.scatter_3d(pcs, x='pc1', y='pc2', z='pc3',color='species')
fig.show()

That's it, i'll be happy to receive your feedbacks :D